In [1]:
%matplotlib notebook

import numpy as np
import tensorflow as tf
import pickle

from matplotlib import pyplot as plt

LEARNING_RATE = 1e-4

data = pickle.load(open('data3_4.p', 'rb'))
test_images, test_labels = pickle.load(open('x_test.p','rb')), pickle.load(open('y_test.p','rb'))

In [85]:
with tf.Session() as sess:    
    
    saver = tf.train.import_meta_graph('/home/ubuntu/notebook/digits-classifier/src/500-256-0.001-model.ckpt.meta')
    saver.restore(sess, '/home/ubuntu/notebook/digits-classifier/src/500-256-0.001-model.ckpt')
    
    g = tf.get_default_graph()
        
    x = g.get_tensor_by_name("input/x-input:0")
    y_ = g.get_tensor_by_name("input/y-input:0")    
    keep_prob = g.get_tensor_by_name("dropout/keep_prob:0")
    y_conv = g.get_collection("y_conv")
    
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    cross_entropy = tf.get_collection('cross_entropy')[0]
    accuracy = tf.get_collection('accuracy')[0]

    images  = test_images.reshape(-1,784)    
    y_data = np.array(test_labels)    

    accuracies = []
    predictions = []
    misclassifieds = []
    n = 40
    col = 8
    row = int(n/col + 0.5) + 1
    
    prediction = sess.run([y_conv], feed_dict = {x: [images[0]], y_: [y_data[0]], keep_prob:1.0})


## Print prediction for first image (This is not consistent?!?)


In [87]:
print("prediction:", np.argmax(prediction[0][0]))

# print(y_data[0])
# plt.imshow(images[0].reshape(28,28))
# plt.show()
# for i in range(10):
#     pred = np.argmax(prediction[i])
#     plt.figure()
#     plt.title(str(pred) + ' correct: ' + str(np.argmax(y_data[i])))
#     plt.imshow(images[i].reshape(28,28),cmap='gray')
#     plt.show()

('prediction:', 0)


## Code that will be useful if model can be loaded

In [ ]:
#     # Feed 100 images at a time into model
#     for i in range(0,len(test_images),100):        
#         end = i+100 if i+100 < len(test_images) else len(test_images) - 1        
#         prediction_vector = sess.run([y_conv], feed_dict = 
#                                      {x: test_images[i:end].reshape(-1,784), y_: labels[i:end], keep_prob: 1.0})                
#         # Add predictions to list.
#         prediction = np.argmax(prediction_vector[0], axis=1)
#         print(len(y_data))
#         for ind, p in enumerate(prediction):
#             print(i, ind, p, y_data[i+ind])
#         predictions.append(prediction)
#         misclassifieds.append(np.equal(prediction, y_data[i:end]))
#     print("prediction", type(prediction))
#     print("actual", y_data[:10],labels)
        
#         print(np.where(misclassified))
#         for j in range(end-i)):        
#             plt.figure()
#             if np.not_equal(predictions[i],labels[i]):            
#                 ax = plt.subplot(1,1,1)
#                 ax.set_title("P: " + str(prediction[i]) + " C: " + str(labels[i]), fontsize=8)
#                 plt.imshow(test_images[i])
#                 plt.show()
#     plt.figure()
#     for i in range(len(images)):
#         if i+1 % 32 == 0:
#             plt.figure()
#             fig = plt.figure(figsize=(10,8))
#         prediction_vector = sess.run([y_conv], feed_dict = 
#                           {x: [images[i]], y_: [labels[i]], keep_prob: 1.0})
#         prediction = np.argmax(prediction_vector[0], axis=1)[0]        
#         if prediction != y_data[i]
#         ax = plt.subplot(row,col,i+1)
#         ax.set_title("P: " + str(prediction) + " C: " + str(y_data[i]), fontsize=8)        
#         plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.4)
#         plt.axis('off')
#         plt.imshow(images[i].reshape(28,28))
    